# Metody Obliczeniowe w Nauce i Technice Laboratorium 6
## Page searcher
### Paweł Gorgolewski

In [ ]:
import numpy as np
import pickle
import wikipedia as wiki

#### Generating wiki dump
Najpierw pobrałem dump indeksów wikipedii ze strony
https://dumps.wikimedia.org/enwiki/20220401/



In [6]:
def get_first_n_titles(n=1000, file='wiki-pages-indexes.txt'):
    titles = []
    with open(file, "r") as f:
        lines = f.readlines()[:n]
        for line in lines:
            titles.append(line.split(sep=':')[-1])

    return titles

def get_articles_content_and_save_pickle(titles):
    for title in titles:
        wiki_page = wiki.page(title)
        wiki_content = wiki_page.content
        #now saving pickle
        pickle.dump(wiki_content, open(title, "wb"))


siemano
<class 'str'>


kod z tej strony
https://www.mediawiki.org/wiki/API%3aAllpages#Python

In [7]:
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "list": "allpages",
    "apfrom": "a",
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

PAGES = DATA["query"]["allpages"]

i = 0
for page in PAGES:
    i+=1
    print(page["title"])

print(i)

A
A!
A!MS
A! (Alexa Feser album)
A! (album)
A"
A"H
A"h
A$
A$$
10
